In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [28]:
def filter_df(df, num_sources=None, num_targets=None):
    """
    A function to filter the data frame by top n sources and targets
    If num_sources or num_targets args are not supplied, they will not be filtered
    """
    if num_targets:
        top_targets = df.sum().sort_values(ascending=False)
        df = df[top_targets[:num_targets].index]
    
    if num_sources:
        top_sources = df.sum(axis=1).sort_values(ascending=False)[:num_sources]
        df = df.loc[top_sources.index]
    
    return df


def create_sankey_df(df, min_val=0):
    """
    Create the human-readable form of the Sankey chart data from an input data frame
    Data can be filtered by a threshold minimum value
    | Source | Source Value | Target | Target Value |
    |    A   |      5       |   i    |      3       |
    |    A   |      5       |   j    |      2       |
    |    B   |      7       |   i    |      1       |
    |    B   |      7       |   k    |      4       |
    """
    
    sources = []
    source_vals = []
    targets = []
    target_vals = []
    for source_name in df.index:
        row = df.loc[source_name]
        sources += [source_name] * sum(row.values > min_val)
        source_vals += [row[row.values > min_val].sum()] * sum(row.values > min_val)
        targets += list(row[row > min_val].index)
        target_vals += list(row[row > min_val].values)
    
    sankey_df = pd.DataFrame({
        'source': sources,
        'target': targets,
        'value': target_vals
    })
    
    return sankey_df


def create_label_dict(node_df, start_idx=0):
    """
    Return a dictionary with labels as keys and indices as values.  Applied 
    to each section of the flow visualization (two nodes and an edge). The
    `node_df` represents each section.
    """
    labels = set(node_df.source).union(node_df.target)
    sorted_labels = sorted(list(labels))
    return {sorted_labels[i]: i + start_idx for i in range(len(sorted_labels))}


def create_final_list(node_df, node_label_dict):
    """
    Return a list of dictionaries and the labels for each section of the flow 
    diagram.
    """
    df = pd.DataFrame({
        "source": node_df.source.map(node_label_dict),
        "target": node_df.target.map(node_label_dict),
        "value" : node_df.value
    })
    
    labels = list(node_label_dict.keys())
    return df.to_dict("records"), labels

In [3]:
investor = pd.read_excel(
    "../data/Equity investor SUP matrix.xlsx",
    engine="openpyxl",
    skiprows=3,
    usecols="B, E:GG",
)

investor = investor.rename(columns={investor.columns[0]: "Ultimate Investor"})

# drop last row because it is a table summary
investor = investor[:-1]
investor = investor.set_index('Ultimate Investor')

In [4]:
financer = pd.read_excel(
    "../data/Financing SUP matrix.xlsx",
    engine="openpyxl",
    skiprows=4,
    usecols="A:AV",
)

# drop last row because it is null
financer = financer[:-1]
financer = financer.set_index('Bank')

In [5]:
producer = pd.read_excel(
    "../data/MFA matrix.xlsx",
    sheet_name="Conversion",
    engine="openpyxl",
    skiprows=1,
    usecols="C:FY",
).dropna()

producer = producer.groupby('Producer').sum()

In [11]:
waste = pd.read_excel(
    "../data/MFA matrix.xlsx",
    sheet_name="Waste",
    engine="openpyxl",
    skiprows=1,
    usecols="B, D:FY",
).dropna()

waste = waste.groupby('Country').sum()

In [43]:
investor_df = create_sankey_df(investor)
financer_df = create_sankey_df(financer)
producer_df = create_sankey_df(producer)
waste_df = create_sankey_df(waste)

In [51]:
# Leaving off investor for now.  This will require a separate, single roll-up 
# JSON object for d3
financer_label_dict = create_label_dict(financer_df)
node1_max = max(financer_label_dict.values())

producer_label_dict = create_label_dict(producer_df, start_idx=(node1_max+1))
node2_max = max(producer_label_dict.values())

waste_label_dict = create_label_dict(waste_df, start_idx=(node2_max+1))

In [52]:
financer_dicts, financer_labels = create_final_list(financer_df, financer_label_dict)
producer_dicts, producer_labels = create_final_list(producer_df, producer_label_dict)
waste_dicts, waste_labels = create_final_list(waste_df, waste_label_dict)

In [56]:
final_d3_object = {
    "links": financer_dicts + producer_dicts + waste_dicts,
    "labels": financer_labels + producer_labels + waste_labels
}

final_d3_object